In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [61]:
df = pd.read_csv(r"C:\\Users\\wijohns\\OneDrive - Microsoft\Desktop\\OnlineRetail\\ItemDeDupe.csv")
random_state = 42


In [74]:
df['InvoiceDateOnly'] = pd.to_datetime(df["InvoiceDateOnly"])
df_agg = pd.DataFrame(df.groupby("InvoiceDateOnly")['Quantity'].sum())

start_date = min(df_agg.index.values)
end_date = max(df_agg.index.values)
t = np.arange(start_date, end_date, np.timedelta64(1, 'D'))

remaining_days = np.setdiff1d(t, df_agg.index.values)
remainder_df = pd.DataFrame(remaining_days)
remainder_df["Quantity"] = 0
remainder_df.columns = ["InvoiceDateOnly", "Quantity"]
remainder_df = remainder_df.set_index("InvoiceDateOnly")

df_dense = pd.concat([df_agg, remainder_df])
df_dense = df_dense.sort_index()

In [76]:
df_dense["lag1"] = df_dense["Quantity"].shift(-1)
df_dense["lag2"] = df_dense["Quantity"].shift(-2)
df_dense["lag7"] = df_dense["Quantity"].shift(-7)
df_dense["lag14"] = df_dense["Quantity"].shift(-14)
df_dense.dropna(inplace=True)

# Training

In [78]:
X = df_dense.drop(["Quantity"], axis=1)
y = df_dense["Quantity"]
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=random_state
    )

In [79]:
model = LinearRegression()
pipe = Pipeline([('scaler', StandardScaler()), ('model', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])

In [81]:
pipe.predict(X_test)

array([ 7345.16993556, 17814.50695994, 21554.18568883, 11909.75968961,
        6414.68908304, 24906.47350648, 13347.07277252,  6903.62668554,
        9907.49821275,  5759.3940001 ,  6617.72485045, 17208.72745334,
        5274.63477266, 12967.72950535,  6091.95789927, 17397.88127882,
       12713.06774891,  7940.77376714,  9140.6778315 ,  7049.47882757,
       12925.42262583,  6957.60751361,  7003.03707274,  5608.92681727,
       11939.91382435,  9203.41149741, 18400.1574295 , 26122.36709083,
        6689.51632598,  9399.46953426, 11276.44385508, 18365.4693235 ,
        9103.44990634, 12017.35467013, 24956.4954348 , 16694.61615295,
       17157.48040498, 12082.85130355, 23564.22573271,  5863.75449379,
       19020.43393209, 16274.63517056,  6565.26918163, 12537.81800091,
       14079.45599345, 26536.68167491, 24128.9330541 ,  8398.30865404,
       16425.40739843,  8403.7059412 ,  8357.62509542, 15654.2329191 ,
       11750.63558699, 12185.261081  , 10356.74481415, 14589.65868185,
      